In [1]:
#!/usr/bin/env python3

import os
import cv2
import darknet
import json
import joblib

darknet.set_gpu(0)
net = darknet.load_net(b"cfg/yolov3.cfg", b"weights/yolov3.weights", 0)
meta = darknet.load_meta(b"cfg/coco.data")

cwd = os.getcwd()
print(cwd)
#print("Running ", __file__, " in ", cwd, ".")

prediction_data_path = "predictions.txt"

root = cwd + "/data/tdata/"

ModuleNotFoundError: No module named 'darknet'

In [2]:
import re
files = os.listdir()
pattern = "(?P<file_name>(?P<base_name>[^_]*_(?P<datetime>(?P<date>\d*)_(?P<time>\d*)))\.pkl$)"
reg = re.compile(pattern)


matches = []

for file in sorted(files):
    match = reg.search(file)
    
    if match is not None:
        matches.append(match)

        
pickled_images = []
for match in matches:
    pickled_images.append(match.groupdict()["base_name"] + ".jpg")
    

    
print(pickled_images)

[]


In [3]:
data_files = os.listdir("data/tdata")

pattern_jpg = "(?P<file_name>(?P<base_name>[^_]*_(?P<datetime>(?P<date>\d*)_(?P<time>\d*)))\.jpg$)"
reg_jpg = re.compile(pattern_jpg)

matches_jpg = []

for data_file in data_files:
    match_jpg = reg_jpg.search(data_file)

    if match_jpg is not None:
        matches_jpg.append(match_jpg)
    
all_images = []
for match_jpg in matches_jpg:
    all_images.append(match_jpg.groupdict()["file_name"])
    
print(all_images)



FileNotFoundError: [Errno 2] No such file or directory: 'data/tdata'

In [4]:
unpickled_images = sorted(set(all_images)-set(pickled_images))
print(unpickled_images)

NameError: name 'all_images' is not defined

In [5]:
result_list = []

for file in unpickled_images:

    # Basename of the image file without the extension
    basename = os.path.splitext(file)[0]

    # Image file
    imagefile = os.path.join(root, file)

    # output text file -> trying to output results in json format
    outfile = basename + "_out.json"

    # output im age name -> darkflow outputs to a png file.
    outimage = basename + "_out.png"

    # Compose the command to be executed - revise later
    # command = "./darknet detect cfg/yolov3.cfg ./yolov3.weights " + \
    #          imagefile + " >> " + prediction_data_path

    # Make an external command - This is where all the problems begin.
    #os.system(command)
    print(imagefile)
    r = darknet.detect(net, meta, imagefile.encode())
    joblib.dump(r, basename + ".pkl")
    result_list.append(r)

    """
    for i in len(r):
        tup = r[i]
        object_type = tup(0)
        prob = tup(1)
        coord = tup(2)
    """    


    # Challenges performing segmentation. Not quite sure how to draw \
    # the bounding boxes on the images. Trying to used opencv for this.
    
    """
    img = cv2.imread(imagefile)
    #for (cls_name, likelihood, (x, y, w, h)) in r:
        # cv2.putText(
        #     img,
        #     cls_name,
        #     (int(x + w/2), int(y + h/2)),
        #     cv2.FONT_HERSHEY_SIMPLEX,
        #     1,
        #     (255, 255, 255)
        # )
        # cv2.rectangle(
        #     img,
        #     (int(x), int(y)),
        #     (int(x + w), int(y + h)),
        #     (255, 255, 255)
        # )

    # Trying to force terminal output in json format
 
    with open(outfile, "w+") as f:
        json.dump([
           {
                "class": cls_name.decode(),
                "likelihood": likelihood
            } for (cls_name, likelihood, _) in r
        ], f)
 

    # cv2.imwrite(outimage, img)
    #cv2.imshow(imagefile, img)

    #tmp_img_path = os.path.join(cwd, "predictions.png")
    #out_img_path = os.path.join(cwd, "predictions", outimage)

    #os.rename(tmp_img_path, out_img_path)

"""



NameError: name 'unpickled_images' is not defined

In [6]:
print(result_list)
#max(result_list, key=lambda x: x[1])
# print(result_list[0:3])
#print(result_list[3][14])

[]


In [ ]:
#Why are you not working?
object_names = []
#object_prob = []

for object_list in result_list:
    for obj in object_list:
        if isinstance(obj,tuple):
            if len(obj) == 3: #Validate that the object has three variables
                name = obj[0]
                #prob = obj[1]
                
                if not (name in object_names): 
                    object_names.append(name)
               # if not (prob in object_prob):
               #     object_prob.append(prob)
                    
#There seem to be a corrupt file        
print(object_names) 

In [ ]:
print(result_list)

In [ ]:
print("These are the detected object Names:\n ",object_names)

number_dict_total = {}
for name in object_names:
    number_dict_total[name] = 0

    
number_dicts_images = []

# result_list: list of length 4; each element is a list of detected_objects

counter = 0
image_counter = 0

for i, detected_object_list in enumerate(result_list):
        
    number_dict_image = {}
    for name in object_names:
        number_dict_image[name] = 0
 
    for tup in detected_object_list:
        if isinstance(tup,tuple):
            if len(tup) == 3:
                name = tup[0]
                if name in object_names:
                    number_dict_image[name] +=1
                    number_dict_total[name] +=1
            
      
    number_dicts_images.append(number_dict_image)
    
print("\nThese are the totals of the detected objects:")
number_dict_total

In [ ]:
#Print the total number of objects detected per image

print("t",number_dict_total)
print("---------------------------------------------------------------------")
for i in range(len(number_dicts_images)):
    print(i, number_dicts_images[i])

In [ ]:
# Try to create atleast one plot from Dictionary data
# Alternatively, try to feed some of the dictionary data into a pandas DF to create visuals. 


In [ ]:
#Try and fix this - Window does not show image. 
for i in range(100000):
    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
    cv2.imshow(imagefile, img)
cv2.destroyAllWindows()

In [ ]:
# Try to plot, but then results are nested lists. 
"""
import matplotlib.pyplot as plt
imlens = []
for i,_im in enumerate(result_list):
    imlens.append(len(_im))
    timefilteredForce = plt.plot(result_list)
    timefilteredForce = pyplot.xlabel('points')
    timefilteredForce = pyplot.ylabel('Force')

plt.show()
imlens
"""
